# Intro to Neural Networks
Adapted from *COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll discuss Neural Networks, which can be used for both Classification and Regression. In particular, we'll discuss 
* perceptrons and multi-layer perceptrons (MLP)
* neural networks with scikit-learn
* how to train a neural network
* intro to TensorFlow

Recommended Reading:
* A. Géron, [Hands-On Machine Learning with Scikit-Learn & TensorFlow](http://proquest.safaribooksonline.com/book/programming/9781491962282) (2017), Ch. 9,10. See also the [associated github page](https://github.com/ageron/handson-ml). 
* Welch Labs, [Neural Networks demystified](https://github.com/stephencwelch/Neural-Networks-Demystified)
* I. Goodfellow and Y. Bengio and A. Courville, [Deep Learning](http://www.deeplearningbook.org/)


In [ ]:
import numpy as np

from sklearn import datasets
from sklearn.datasets import load_iris, make_moons, load_breast_cancer, fetch_openml, fetch_california_housing
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Neural Networks and deep learning

[Artificial Neural Networks](https://en.wikipedia.org/wiki/Artificial_neural_network) were originally motivated by the brain, which is composed of a network of neurons. Each neuron recieves a (chemical) signal from other neurons, does a small computation and then decides if and how to release more chemicals. This composition of small calculations can perform complicated tasks! Similiarly, an artificial neural network is a network composed of neurons, which we simply think of as a computational units. 

Large scale neural networks are at the core of [deep learning](https://en.wikipedia.org/wiki/Deep_learning), which has gained much publicity for performing very impressive machine learning tasks in the past few years, such as, 
* classifying billions of images (*e.g.*, Google Images)
+ speech recognition (*e.g.*, Amazon's Alexa or Apple’s Siri)
+ video recommendation (*e.g.*, YouTube), 
+ beating the world champion at the game of Go (DeepMind’s AlphaGo).

Neural Networks can generally be used for supervised learning tasks, such as classification and regression. 

## Perceptrons

The simplest element in a neural network is called the [perceptron](https://en.wikipedia.org/wiki/Perceptron). The *perceptron* is an binary classifier. It maps a real input $x \in \mathbb R^m$ to a binary output
$$
f(x) = 
\begin{cases}
1 & \textrm{if } \ w\cdot x + b > 0 \\
0 & \textrm{otherwise}
\end{cases}.
$$
Here, $w \in \mathbb R^m$ is a vector of weights and $b$ is a scalar called the *bias*. (This is very similiar to the binary classifier we saw when looking at support vector machines.) 

It is customary to represent this function by the following diagram. 

<img src="Perceptron.svg" title="https://commons.wikimedia.org/wiki/File:Perceptron.svg#/media/File:Perceptron.svg" width="500">

Just like for previous classification methods, we first *train* the network on data, which is to say that we find good choices of $w$ and $b$ for our application. Then we use the neural network to classify new data points. 

Of course, a single perceptron is only a linear discriminator (similiar to logistic regression and linear SVM). But things become much more interesting when you start composing many neurons, that is, considering networks with more *layers*. 

The way in which we put together the neurons is referred to as the **network architecture**. There are many ways to do this. Here is a peak at the [neural-network-zoo](http://www.asimovinstitute.org/neural-network-zoo/): 
<img src="neuralnetworks.png" title="http://www.asimovinstitute.org/neural-network-zoo/" 
width="700">

There are many different network architectures. The most important thing about all of these neural networks is that there is an *input layer*, typically drawn on the left hand side and an *output layer*, typically drawn on the right hand side. The middle layers are sometimes called *hidden layers*. 

<img src="Colored_neural_network.svg" title="https://en.wikipedia.org/wiki/Artificial_neural_network#/media/File:Colored_neural_network.svg" 
width="500">




In all of these neural network designs, each layer has its own weight vectors and biases that need to be trained. Consequently, *training* a neural network is a much harder job than we have seen for previous methods. It also requires more data. 

## Neural Networks in Practice

In the past, we have use scikit-learn for classification. Scikit-learn also has Neural Network library [here](http://scikit-learn.org/stable/modules/neural_networks_supervised.html). However, this implementation does not scale to large-scale applications (no GPU support or deep learning architectures). 

There are many other packages that have more complete implementations of neural networks. Here is a partial list with short statements taken from the packages.
* [Pytorch](https://pytorch.org): Seems to have knocked off Tensoflow as the top choice for most NN researchers/developers.  We'll us it in this course.
* [TensorFlow](https://github.com/tensorflow/tensorflow): TensorFlow™ is an open source C++ software library for numerical computation using data flow graphs. Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors) communicated between them. The flexible architecture allows you to deploy computation to one or more CPUs or GPUs in a desktop, server, or mobile device with a single API. 
+ [Torch](http://torch.ch/): Torch is a scientific computing framework with wide support for machine learning algorithms that puts GPUs first.
+ [CNTK](https://github.com/Microsoft/cntk)  Cognitive Toolkit (CNTK) is an open source deep-learning toolkit developed by Microsoft. 
+ [Theano](https://github.com/Theano/Theano): Theano is a Python library that allows you to define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently. It can use GPUs and perform efficient symbolic differentiation.

+ [keras](https://github.com/keras-team/keras): Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano.
+ [MXNet](https://github.com/dmlc/mxnet): Lightweight, Portable, Flexible Distributed/Mobile Deep Learning with Dynamic, Mutation-aware Dataflow Dep Scheduler; for Python, R, Julia, Scala, Go, Javascript and more
+ [Caffe](http://caffe.berkeleyvision.org/): Caffe is a deep learning framework made with expression, speed, and modularity in mind. It is developed by Berkeley AI Research (BAIR) and by community contributors.
+ [Lasagne](https://github.com/Lasagne/Lasagne): Lightweight library to build and train neural networks in Theano. 
+ [prettytensor](https://github.com/google/prettytensor/): 
Pretty Tensor provides a high level builder API for TensorFlow. It provides thin wrappers on Tensors so that you can easily build multi-layer neural networks.

+ [Deeplearning4j](https://deeplearning4j.org/): Open-Source, Distributed, Deep Learning Library for the JVM
+ [H2O](https://github.com/h2oai): Fast Scalable Machine Learning For Smarter Applications

We'll start with the scikit-learn implementation, since this environment is familiar and then use TensorFlow later. 



## Neural networks with scikit-learn

scikit-learn has a few different neural network functions:
1. [perceptron](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html)
1. [multi-layer perceptron (MLP) classifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)
1. [multi-layer perceptron (MLP) regressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)

The scikit-learn user guide for supervised learning using neural networks is [here](http://scikit-learn.org/stable/modules/neural_networks_supervised.html). 

Let's first test the `MLPClassifier` on the [two moons dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html). 

In [ ]:
# X contains two features
# y contains labels
X,y = make_moons(n_samples=1000,random_state=1,noise=0.2)
X = StandardScaler().fit_transform(X)

# Plot the data, color by class
plt.scatter(X[y == 1, 0], X[y == 1, 1], color="DarkBlue", marker="s",label="class 1")
plt.scatter(X[y == 0, 0], X[y == 0, 1], color="DarkRed", marker="o",label="class 2")
plt.legend(scatterpoints=1)
plt.title('Two Moons Dataset')
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.show()

In [ ]:
model = MLPClassifier( hidden_layer_sizes=(3,3,3), max_iter=1000, alpha=1e-4,
                       solver='adam', verbose=10, random_state=1, 
                       learning_rate_init=.1 )
model.fit( X, y ) # What are X and y here?

In [ ]:
# Plot the data, color by class
plt.scatter(X[y == 1, 0], X[y == 1, 1], color="DarkBlue", marker="s",label="class 1")
plt.scatter(X[y == 0, 0], X[y == 0, 1], color="DarkRed", marker="o",label="class 2")
plt.legend(scatterpoints=1)

# Plot the predictions made by NN
x_min, x_max = X[:,0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),np.linspace(y_min, y_max, 200))
zz = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.contourf(xx, yy, zz, cmap=ListedColormap(['DarkRed', 'DarkBlue']), alpha=.2)
plt.contour(xx, yy, zz, colors="black", alpha=1, linewidths=0.2) 

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.title('Classification of Two Moons using MLPClassifier')
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.show()

In [ ]:
print('After ', model.n_iter_, ' iterations, the loss is ', model.loss_)
print('model coef shapes')
[print(coef.shape) for coef in model.coefs_]
print('model coefs')
[print(coef) for coef in model.coefs_]
print('model intercepts')
[print(coef) for coef in model.intercepts_]

print(model.get_params())


There are a lot more function parameters for [`MLPClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) than for other scikit-learn classification methods. You'll find that tweaking them also makes a very big difference in the output. Here are some of the important parameters:

#### Network architecture parameters

+ **hidden_layer_sizes**: tuple, length = n_layers - 2, default (100,). 
The ith element represents the number of neurons in the ith hidden layer.

+ **activation**: {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’. 
Activation function for the hidden layer.
 - ‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
 - ‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
 - ‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
 - ‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

#### Optimization related parameters
+ **solver**: {‘lbfgs’, ‘sgd’, ‘adam’}, default ‘adam’. 
The solver for weight optimization.
 - ‘lbfgs’ is an optimizer in the family of quasi-Newton methods.
 - ‘sgd’ refers to stochastic gradient descent.
 - ‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

+ **alpha**: float, optional, default 0.0001.
L2 penalty (regularization term) parameter.

+ **max_iter**: int, optional, default 200. 
Maximum number of iterations. The solver iterates until convergence (determined by ‘tol’) or this number of iterations. For stochastic solvers (‘sgd’, ‘adam’), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps.

+ **random_state**: int, RandomState instance or None, optional, default None. 
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

+ **tol**: float, optional, default 1e-4. 
Tolerance for the optimization. When the loss or score is not improving by at least tol for two consecutive iterations, unless learning_rate is set to ‘adaptive’, convergence is considered to be reached and training stops.

+ **verbose**: bool, optional, default False. Whether to print progress messages to stdout.



**Exercise**: By chainging the hidden_layer_sizes, activation function, and the random_state, see if you can find a better classification of the above two moons dataset. 


## Multi-layer Perceptrons

Here, we take a closer look at Multi-layer Perceptrons; this material is taken from the [scikit-learn user guide](http://scikit-learn.org/stable/modules/neural_networks_supervised.html)

Given a set of features $X = \{x_1, x_2, ..., x_n\}$ and a target $y$, a **Multi-layer Perceptron** can learn a non-linear function for either classification or regression. Here is an example with one hidden layer. 
 
<img src="multilayerperceptron_network.png" title="http://scikit-learn.org/stable/_images/multilayerperceptron_network.png" 
width="300">

Each neuron applies an affine transformation 
$$
x \mapsto w\cdot x + b
$$
and then a non-linear *activation function* $g\colon \mathbb R \to \mathbb R$. The composition looks like:
$$
x \mapsto g(w\cdot x + b).
$$
The output layer receives the values from the last hidden layer and transforms them into output values.

In the previous example, we used saw the trained cofficients by printing `model.coefs_` and `model.intercepts_`. 

There are several choices of **activation function**: hyperbolic tangent, logistic, and rectified linear unit (ReLU). We used the default activiation function, ReLU. 


In [ ]:
# see Géron, Ch. 10

def logit(z):
    return 1 / (1 + np.exp(-z))

def relu(z): # Rectified Linear Unit
    return np.maximum(0, z)

def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)

z = np.linspace(-5, 5, 200)

plt.figure(figsize=(11,4))

plt.subplot(121)
plt.plot(z, np.sign(z), "r-", linewidth=2, label="Step")
plt.plot(z, logit(z), "g--", linewidth=2, label="Logit")
plt.plot(z, np.tanh(z), "b-", linewidth=2, label="Tanh")
plt.plot(z, relu(z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
plt.legend(loc="center right", fontsize=14)
plt.title("Activation functions", fontsize=14)
plt.axis([-5, 5, -1.2, 1.2])

plt.subplot(122)
plt.plot(z, derivative(np.sign, z), "r-", linewidth=2, label="Step")
plt.plot(0, 0, "ro", markersize=5)
plt.plot(0, 0, "rx", markersize=10)
plt.plot(z, derivative(logit, z), "g--", linewidth=2, label="Logit")
plt.plot(z, derivative(np.tanh, z), "b-", linewidth=2, label="Tanh")
plt.plot(z, derivative(relu, z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
#plt.legend(loc="center right", fontsize=14)
plt.title("Derivatives", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])

plt.show()

## Training a neural network
We'll follow the scikit-learn  [user guide](http://scikit-learn.org/stable/modules/neural_networks_supervised.html) 
to see how the multi-layer perceptron (MLP) neural network is trained. 

The MLP uses a loss function of the form 
$$
Loss(\hat{y},y,W) =  \frac{1}{2} \sum_{i=1}^n f(\hat{y}_i(W),y_i) + \frac{\alpha}{2} \|W\|_2^2
$$
Here, 
+ $y_i$ are the labels for the $i$-th example, 
+ $\hat{y}_i(W)$ are the predicted label for the $i$-th example, 
+ $f$ is a function that measures the error, typically $L^2$ difference for regression or cross-entropy for classification, and 
+ $\alpha$ is a regularization parameter. 

Starting from initial random weights, the loss function is minimized by repeatedly updating these weights. The details of this depend on the chosen method, either a quasi-Newton method `lbfgs`, stochastic gradident descent `sgd`, or `adam`. 

In the **gradient descent method**, the gradient $\nabla_{W} Loss$ of the loss with respect to the weights is computed. The weights are then changed in the negative gradient direction using a step-length or learning-rate $\varepsilon>0$: 
$$
W \leftarrow W - \varepsilon \nabla_W {Loss}.
$$
The algorithm stops when it reaches a preset maximum number of iterations, `max_iter`, 
or when the improvement in loss is below a preset small number, `tol`.

The gradient of $W$ is simply computed using the chain rule from calculus. In principle the idea is simple, but in practice it is a complicated job. Data analysts have figured out a clever way how to organize this calculation. This is sometimes called *back propagation*. 

A complete description of `lbfgs`, `sgd`, and `adam` is beyond the scope of the course. I'll just say that they are clever modifications to the gradient descent method. 

Let's see a comparison of optimization methods, taken from [this page](http://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_training_curves.html). 

In [ ]:
# different learning rate schedules and momentum parameters
params = [{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': 0, # SGD - Stochastic Gradient Descent
           'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': 0,
           'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'adam', 'learning_rate_init': 0.01}]

labels = ["constant learning-rate", "constant with momentum",
          "constant with Nesterov's momentum",
          "inv-scaling learning-rate", "inv-scaling with momentum",
          "inv-scaling with Nesterov's momentum", "adam"]

plot_args = [{'c': 'red', 'linestyle': '-'},
             {'c': 'green', 'linestyle': '-'},
             {'c': 'blue', 'linestyle': '-'},
             {'c': 'red', 'linestyle': '--'},
             {'c': 'green', 'linestyle': '--'},
             {'c': 'blue', 'linestyle': '--'},
             {'c': 'black', 'linestyle': '-'}]


def plot_on_dataset(X, y, ax, name):
    # for each dataset, plot learning for each learning strategy
    print("\nlearning on dataset %s" % name)
    ax.set_title(name)
    X = MinMaxScaler().fit_transform(X)
    mlps = []
    if name == "digits":
        # digits is larger but converges fairly quickly
        max_iter = 15
    else:
        max_iter = 400

    for label, param in zip(labels, params):
        print("training: %s" % label)
        mlp = MLPClassifier(verbose=0, random_state=0,
                            max_iter=max_iter, **param)
        mlp.fit(X, y)
        mlps.append(mlp)
        print("Training set score: %f" % mlp.score(X, y))
        print("Training set loss: %f" % mlp.loss_)
    for mlp, label, args in zip(mlps, labels, plot_args):
            ax.plot(mlp.loss_curve_, label=label, **args)


fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# load / generate some toy datasets
iris = datasets.load_iris()
digits = datasets.load_digits()
data_sets = [(iris.data, iris.target),
             (digits.data, digits.target),
             datasets.make_circles(noise=0.2, factor=0.5, random_state=1),
             datasets.make_moons(noise=0.3, random_state=0)]

for ax, data, name in zip(axes.ravel(), data_sets, ['iris', 'digits',
                                                    'circles', 'moons']):
    plot_on_dataset(*data, ax=ax, name=name)

fig.legend(ax.get_lines(), labels, ncol=3, loc="upper center")
plt.show()

Some advise on optimization methods according to [this page](http://scikit-learn.org/stable/modules/neural_networks_supervised.html): 
* Empirically, we observed that L-BFGS converges faster and with better solutions on small datasets. For relatively large datasets, however, Adam is very robust. It usually converges quickly and gives pretty good performance. SGD with momentum or nesterov’s momentum, on the other hand, can perform better than those two algorithms if learning rate is correctly tuned.

## Example: breast cancer data

In [ ]:
cancer = load_breast_cancer()
print(cancer.keys())

# 569 data points with 30 features
cancer['data'].shape

# full description:
print(cancer['DESCR'])

X = cancer['data']
y = cancer['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

scaler = StandardScaler()
scaler.fit(X_train) # Fit only to the training data

# Apply scaling to data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30),random_state=1)
mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)
print(confusion_matrix(y_test,y_pred))

## Example: MNIST dataset

Let's train a multi layer perceptron on the MNIST dataset. 

In [ ]:
from scipy.io import loadmat
import urllib
mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
mnist_path = "./mnist-original.mat"
response = urllib.request.urlopen(mnist_alternative_url)
with open(mnist_path, "wb") as f:
    content = response.read()
    f.write(content)
mnist_raw = loadmat(mnist_path)
mnist = {
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0],
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
}

# rescale the data, use the traditional train/test split
X, y = mnist['data'] / 255., mnist['target']
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400, alpha=1e-4,
                     solver='adam', verbose=10, tol=1e-4, random_state=1)
mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

y_pred = mlp.predict(X_test)
print(confusion_matrix(y_test,y_pred))

**Exercise**: By adjusting the parameters in the MLPClassifier, imporve the test set score. 
    

**Note**: [This webpage](http://scikit-learn.org/stable/auto_examples/neural_networks/plot_mnist_filters.html)
tries to interpret the MLP classification weights learned for the MNIST dataset. 



## Example: Regression with Neural Networks in scikit-learn

Let's use a multi-layer perceptron for regression. This can be done with the scikit-learn [`MLPRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) function. 


In [ ]:
housing = fetch_california_housing()

print(housing.keys())

# 20640 data points with 8 features
housing['data'].shape

# full description:
print(housing['DESCR'])

X = housing['data']
y = housing['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

# scale data
scaler = StandardScaler()
scaler.fit(X_train) # Fit only to the training data

# Apply scaling to data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Linear Regression with Scikit-Learn
lin_reg = LinearRegression()
print(lin_reg.get_params())

lin_reg.fit(X_train, y_train)
print(lin_reg.intercept_)
print(lin_reg.coef_)

print(lin_reg.score(X_test,y_test)) # score = 1 is good

In [ ]:
# MLP regression with Scikit-Learn
mlp_reg = MLPRegressor(hidden_layer_sizes=(8,8),verbose=0,random_state=2,solver='adam')
print(mlp_reg.get_params())

mlp_reg.fit(X_train, y_train)

print(mlp_reg.score(X_test,y_test)) # score = 1 is good

As mentioned, using the "identity" activation function (f(y) = y) even a complex neural network gives the same result as linear regression!  We'll try that below...

In [ ]:
# MLP regression with Scikit-Learn with identity 
mlp_reg = MLPRegressor(hidden_layer_sizes=(8,8),verbose=0,random_state=2,solver='adam', activation='identity')
print(mlp_reg.get_params())

mlp_reg.fit(X_train, y_train)

print(mlp_reg.score(X_test,y_test)) # score = 1 is good



## Neural Networks demystified

There is a good sequence of 7 videos called **Neural Networks demystified** from Welch Labs that builds and trains a neural network *from scratch* in python. 
* Part 1: [Data + Architecture](https://www.youtube.com/watch?v=bxe2T-V8XRs)
* Part 2: [Forward Propagation](https://www.youtube.com/watch?v=UJwK6jAStmg)
* Part 3: [Gradient Descent](https://www.youtube.com/watch?v=5u0jaA3qAGk)
* Part 4: [Backpropagation](https://www.youtube.com/watch?v=GlcnxUlrtek)
* Part 5: [Numerical Gradient Checking](https://www.youtube.com/watch?v=pHMzNW8Agq4&t=22s)
* Part 6: [Training](https://www.youtube.com/watch?v=9KM9Td6RVgQ)
* Part 7: [Overfitting, Testing, and Regularization](https://www.youtube.com/watch?v=S4ZUwgesjS8)

If you're interested in learning more about how Neural Networks are trained, I would recommend watching these videos. 

The accompanying code is [on github](https://github.com/stephencwelch/Neural-Networks-Demystified) and can be obtained via 
```
git clone https://github.com/stephencwelch/Neural-Networks-Demystified.git
```